In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

<h1>Question 1 :</h1>
<h2>Q1 : 1</h2>
Lecture et presentation de statistiques (DPAE)

In [ ]:
TD = pd.read_excel('Tableau_Declarations-Prealables-Embauche.xlsx',header=0)
TD.head()

In [ ]:
# Extraction des CDD et presentation des statistiques

mask_CDD = TD['Nature_de_contrat'].str.match("CDD.*")
CDD=TD[mask_CDD]
 
rng = pd.date_range('2000 Jan 1', periods=len(CDD), freq='MS')
CDD.index = rng
CDD.describe()


In [ ]:
# Extraction des CDI et presentation des statistiques

mask_CDI = TD['Nature_de_contrat'].str.match("CDI.*")
CDI=TD[mask_CDI]
rng = pd.date_range('2000 Jan 1', periods=len(CDI), freq='MS')
CDI.index = rng
CDI.describe()

In [ ]:
# Pour CDI : Representation du DPAE_brut et de l'evolution de sa moyenne
ax= CDI.plot(y=["DPAE_brut"],style='-')
CDI.resample('BA').mean().plot(y=["DPAE_brut"],style=':',ax=ax,figsize=(20,8))
plt.legend(["DPAE_brut","mean of DPAE_brut"])

CDI.boxplot(column=["DPAE_brut"], by='Annee', figsize=(20,8),showmeans=True)
CDI.boxplot(column=["DPAE_brut"], by='Trimestre', figsize=(10,8),showmeans=True)


In [ ]:
# Pour CDD : Representation du DPAE_brut et de l'evolution de sa moyenne
ax= CDD.plot(y=["DPAE_brut"],style='-')
CDD.resample('BA').mean().plot(y=["DPAE_brut"],style=':',ax=ax,figsize=(20,8))
plt.legend(["DPAE_brut","mean of DPAE_brut"])

CDD.boxplot(column=["DPAE_brut"], by='Annee', figsize=(20,8))
CDD.boxplot(column=["DPAE_brut"], by='Trimestre', figsize=(10,8))


<h2>Question 1 : 2</h2>
<h3>Q1 : 2 : 1</h3>
Nettoyage de données 

In [ ]:
EES = pd.read_excel('Tableau_Emploi-Salarie-Trimestre-National_09032022.xlsx',index_col=0)

#EES.head(5)

In [ ]:
#On interprete les (O) comme des NA
EES_A = EES.replace("(O)",pd.NA)

#EES_A.head()

In [ ]:
#On retire des colonnes jugées non pertinentes
EES_NET = EES_A.drop(["idBank","Dernière mise à jour", "Période"],axis=1)

#EES_NET.head(5)

In [ ]:
#On supprime les secteurs comportant des valeurs NA
EES_NET=EES_NET.dropna(axis=0)

#EES_NET.head(5)

In [ ]:
#On transpose le DF pour avoir les dates commes index , et les secteurs comme colonnes
EES_NET=EES_NET.T.copy()

EES_NET.head(5)

<h3>Q1 : 2 : 2</h3>
Création d'une seule série temporelle qui somme l’ensemble de secteurs

In [ ]:
#On ajoute un ligne en bas du DF qui represente le Total de chaque colonne
EES_NET['total'] = EES_NET.sum(numeric_only=False, axis=1)
EES_NET['total']

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(EES_NET['total'])
plt.xlabel("Date")
plt.ylabel("Sum of all fields")
plt.title("Plot")

plt.show()

<p>Observation : On remarque là une hausse des Emplois salariés de trimestre en trimestre suivant les années</p>

<h3>Q1 : 2 : 3</h3>
Etude de 2-5 secteurs :<br>
Dans notre cas, on en traite 4

In [ ]:
#Selection de 4 secteurs (qui sont les 4 premières colonnes)
EES_NET[EES_NET.columns[:4]]

In [ ]:
secteur1 = EES_NET[EES_NET.columns[0]].astype(float)
secteur2 = EES_NET[EES_NET.columns[1]].astype(float)
secteur3 = EES_NET[EES_NET.columns[2]].astype(float)
secteur4 = EES_NET[EES_NET.columns[3]].astype(float)

secteurs=[secteur1,secteur2,secteur3,secteur4]


In [ ]:
#Plot par secteur
for secteur in secteurs :
    plt.figure(figsize=(20, 8))
    plt.plot(secteur.index, secteur.values)
    plt.xlabel("Date")
    plt.ylabel("Emplois")
    plt.title(secteur.name)
    plt.show()

Dans notre cas on remarque que :
<ul>
    <li>Le premier secteur presente une baisse au niveau du nombre d'Emplois salariés trimestriels</li>
    <li>Le deuxieme secteur monte et descend, mais par rapport aux année 1970-1980, les valeurs des dernières années sont toutes inferieurs</li>
    <li>Le troisième et le dernier secteur montrent une hausse du nombre d'Emplois salariés trimestriels au fil des années</li>
</ul>

<h3>Q1 : 2 : 4</h3>
Statistiques classiques 

In [ ]:
#Description des secteurs
for secteur in secteurs :
    print(f"\npour {secteur.name} :\n\n{secteur.describe()}")

<h1>Question 2 :</h1>
<h2>Q2 : 1</h2>
Evaluation de la saisonnalité sur DPAE

In [ ]:
#Saisonalité des CDD
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(CDD["DPAE_brut"], period=12) 

plt.figure(figsize=(25,8))
plt.plot(CDD.index, CDD["DPAE_brut"], 'k')
plt.legend(['Cases'])
plt.figure(figsize=(25,8))
plt.plot(CDD.index,decomposition.trend)
plt.plot(CDD.index,decomposition.seasonal)
plt.plot(CDD.index,decomposition.resid)
plt.legend(['Trend', 'Seasonal', 'Resid'])

In [ ]:
#Saisonalité des CDI
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(CDI["DPAE_brut"], period=12)

plt.figure(figsize=(25,8))
plt.plot(CDI.index, CDI["DPAE_brut"], 'k')
plt.legend(['Cases'])
plt.figure(figsize=(25,8))
plt.plot(CDI.index,decomposition.trend)
plt.plot(CDI.index,decomposition.seasonal)
plt.plot(CDI.index,decomposition.resid)
plt.legend(['Trend', 'Seasonal', 'Resid'])


Observation : On peut clairement voir que le CDI et CDD suivent une saisonnalité

<h2>Q2 : 2</h2>
Evaluation de la saisonnalité sur DPAE

In [ ]:
#On representera Trend et (Seasonal,Resid) separement vu la difference d'echelles. On aura ainsi une meilleur visibilité des données

for secteur in secteurs :
    decomposition = seasonal_decompose( secteur.values,period=4)
    #Emplois
    plt.figure(figsize=(25,8))
    plt.plot(secteur.index,secteur.values , 'k')
    plt.title("Emplois du secteur "+secteur.name)
    plt.show()

    # trend
    plt.figure(figsize=(25,8))
    plt.plot(secteur.index,decomposition.trend)
    plt.title("Tendance du secteur "+secteur.name)
    plt.show()

    #seasonal + resid
    plt.figure(figsize=(25,8))
    plt.title("Secteur "+secteur.name)
    plt.plot(secteur.index,decomposition.seasonal)
    plt.plot(secteur.index,decomposition.resid)
    plt.legend(['Seasonal', 'Resid'])
    plt.show()

Observation : On peut clairement voir que les valeurs de secteurs ne suivent pas une saisonnalité

<h2>Q2 : 3</h2>
Calcule de la corrélation entre les séries de DPAE et les séries sélectionnées d’EESSA

In [ ]:
#On preleve les valeurs prisent entre 2000-2021
CDD_F=CDD[(CDD.index >= pd.Timestamp('2000')) & (CDD.index < pd.Timestamp('2022'))]

#On calcule une moyenne pour chaque 3 mois successifs
#pour representer la valeur du trimestre pour cette année
dd=pd.DataFrame(columns=list(['Trimestre',"DPAE_brut"]))
for i in range(4*(2022-2000)):
    s=0
    for j in range(3):
        s+=CDD_F.iloc[i*3+j]["DPAE_brut"]
    s/=3
    a=pd.DataFrame(data={"Trimestre" : [str(CDD_F.iloc[i*3]["Annee"])+"-T"+str(CDD_F.iloc[i*3]["Trimestre"])] ,"DPAE_brut":[s]})
    dd = pd.concat([dd , a])
dd.index = dd["Trimestre"]
dd=dd["DPAE_brut"].astype(float)
dd

In [ ]:
#Ramener les secteurs dur la periode 2000-2021

#ici on commence avec 1970-T4, et pour chaque année en a 4 valeurs,
#pour arriver au debut des valeurs de 2000 on declade de 4*29 + 1 
secteur1_c = secteur1 [4*29 + 1 :]
secteur2_c = secteur2 [4*29 + 1 :]
secteur3_c = secteur3 [4*29 + 1 :]
secteur4_c = secteur4 [4*29 + 1 :]

secteur_c=[secteur1_c,secteur2_c,secteur3_c,secteur4_c]

secteur1_c

In [ ]:
######################## Correlation avec CDD ####################################
for secteur in secteur_c:
    print(f"\nCorrelation entre CDD et {secteur.name} :\n\t",
    np.corrcoef(secteur.values,dd.values)[0,1])

<h3>Interpretation :</h3>
<ul>
    <li>CDD et Emplois salariés trimestriels - Industrie - Sections B, C, D et E - CVS :</li>
    <p>On a un degre de correlation qui est bas (Low degree Correlation)</p>
    <li>CDD et Emplois salariés trimestriels - Construction - Section F - CVS :</li>
    <p>On a un degre de correlation qui est bas (Low degree Correlation)</p>
    <li>CDD et Emplois salariés trimestriels - Tertiaire marchand - Sections G, H, I, J, K, L, M, N, R, S, T - CVS :</li>
    <p>On a un degre de correlation qui est bas (Low degree Correlation)</p>
    <li>CDD et Emplois salariés trimestriels - Industries extractives, énergie, eau, gestion des déchets et dépollution - A17 : DE - CVS :</li>
    <p>On a un degre de correlation qui est bas (Low degree Correlation)</p>
</ul>

In [ ]:
######################## Correlation avec CDI ####################################
#On deduit un DataFrame de forme adequate
CDI_F=CDI[(CDI.index >= pd.Timestamp('2000')) & (CDI.index < pd.Timestamp('2022'))]
di=pd.DataFrame(columns=list(['Trimestre',"DPAE_brut"]))
for i in range(4*(2022-2000)):
    s=0
    for j in range(3):
        s+=CDI_F.iloc[i*3+j]["DPAE_brut"]
    s/=3
    a=pd.DataFrame(data={"Trimestre" : [str(CDI_F.iloc[i*3]["Annee"])+"-T"+str(CDI_F.iloc[i*3]["Trimestre"])] ,"DPAE_brut":[s]})
    di = pd.concat([di , a])
di.index = di["Trimestre"]
di=di["DPAE_brut"].astype(float)

for secteur in secteur_c:
    print(f"\nCorrelation entre CDI et {secteur.name} :\n\t",
    np.corrcoef(secteur.values,di.values)[0,1])

<h3>Interpretation :</h3>
<ul>
    <li>CDI et Emplois salariés trimestriels - Industrie - Sections B, C, D et E - CVS :</li>
    <p>On a un degre modéré de correlation (Moderate degree correlation)</p>
    <li>CDI et Emplois salariés trimestriels - Construction - Section F - CVS :</li>
    <p>On a un degre de correlation qui est bas (Low degree Correlation)</p>
    <li>CDI et Emplois salariés trimestriels - Tertiaire marchand - Sections G, H, I, J, K, L, M, N, R, S, T - CVS :</li>
    <p>On a un haut degre de correlation (High degree correlation)</p>
    <li>CDI et Emplois salariés trimestriels - Industries extractives, énergie, eau, gestion des déchets et dépollution - A17 : DE - CVS :</li>
    <p>On a un degre modéré de correlation (Moderate degree correlation)</p>
</ul>

<h1>Question 3</h1>
Séparation des données en ensemble d’apprentissage et de test en vu d'effectuer des prédictions par lissage temporel

In [ ]:
#On recupère le DPAE_brut, et on separe notre dataset en 80/20 pour generer le training et test sets
CDD = CDD["DPAE_brut"]
CDD_train = CDD[:212]
CDD_test = CDD[212:]
CDD_test

In [ ]:
#Comme pour le CDD, on recupère le DPAE_brut, et on separe notre dataset en 80/20 pour generer le training et test sets
CDI = CDI["DPAE_brut"]
CDI_train = CDI[:212]
CDI_test = CDI[212:]



In [ ]:
#secteur_train contiendra les training sets des 4 secteurs, et secteur_test les test sets.
secteur_train = []
secteur_test = []
for secteur in secteurs :
    secteur_train.append(secteur[:164])
    secteur_test.append(secteur[164:])

  

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, Holt

#Lissage exponentiel pour CDD
fit3 = ExponentialSmoothing(
    CDD_train,
    seasonal_periods = 12,
    trend = "add",
    seasonal = "add",
    use_boxcox = False,
    initialization_method = "estimated",
).fit()

In [ ]:
#Prediction CDD
# Noir      : training set
# Blue      : test set
# Yellow    : prediction

forecast_CDD = fit3.forecast(len(CDD_test))
full_curve = np.concatenate((fit3.fittedvalues, forecast_CDD))

plt.figure(figsize=(25,8))

plt.plot(CDD_train.index,CDD_train.values, marker="o", color="black", label='CDD_Train')
plt.plot(CDD_test.index,CDD_test.values, marker="o", color="blue", label='CDD_Test')
plt.plot(CDD.index,full_curve, marker="o", color="orange", label='Forecast')
plt.legend()
plt.show()

In [ ]:
#Lissage pour CDI
fit3 = ExponentialSmoothing(
    CDI_train,
    seasonal_periods = 12,
    trend = "add",
    seasonal = "add",
    use_boxcox = False,
    initialization_method = "estimated",
).fit()

In [ ]:
#Prediciton CDI
forecast_CDI = fit3.forecast(len(CDI_test))
full_curve = np.concatenate((fit3.fittedvalues, forecast_CDI))

plt.figure(figsize=(25,8))

plt.plot(CDI_train.index,CDI_train.values, marker="o", color="black", label='CDI_Train')
plt.plot(CDI_test.index,CDI_test.values, marker="o", color="blue", label='CDI_Test')
plt.plot(CDI.index,full_curve, marker="o", color="orange", label='Forecast')
plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
#Lissage par secteur en prenant en consideration leurs non saisonalité
fit3 = []
for secteur in range(4) :
    fit3.append( Holt(
        secteur_train[secteur],
        initialization_method = "estimated",
    ).fit( smoothing_level=0.2, smoothing_trend=0.2) )

In [ ]:
#Prediction et affichage d'un graph representatif
# Noir      : training set
# Blue      : test set
# Yellow    : prediction

forecast = []
for secteur in range(4) :
    forecast.append(fit3[secteur].forecast(len(secteur_test[secteur])))
    full_curve = np.concatenate((fit3[secteur].fittedvalues, forecast[secteur]))

    plt.figure(figsize=(25,8))

    plt.plot(secteur_train[secteur].index,secteur_train[secteur].values, marker="o", color="black", label="Train data")
    plt.plot(secteur_test[secteur].index,secteur_test[secteur].values, marker="o", color="blue", label='Test data')
    plt.plot(secteurs[secteur].index,full_curve, marker="o", color="orange", label='Forecast')
    plt.title(secteurs[secteur].name)
    plt.legend()
    plt.show()

<h1>Question 4 :</h1>
Évaluation des prédictions par le calcul d’erreurs.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt


rmse_CDD=sqrt(mean_squared_error(CDD_test, forecast_CDD))
print("RMSE_CDD\n\t= ",rmse_CDD)


rmse_CDI=sqrt(mean_squared_error(CDI_test, forecast_CDI))
print("\nRMSE_CDI\n\t= ",rmse_CDI)


rmse_=[]
for secteur in range(4) :
    rmse_.append(sqrt(mean_squared_error(secteur_test[secteur], forecast[secteur])))
    print("\nRMSE "+secteurs[secteur].name,"\n\t= ",rmse_[secteur])

<h1>Question 5 :</h1>
Prediction en utilisant le modele ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(CDD_train, order=(2,1,2))
CDD_ARIMA_model = model.fit()
forecast_CDD_ARIMA=CDD_ARIMA_model.forecast(len(CDD_test))
CDD_results = pd.concat([CDD_ARIMA_model.fittedvalues,forecast_CDD_ARIMA])
plt.figure(figsize=(25,8))
plt.plot(CDD_train, marker="o", color="black", label="CDD_train")
plt.plot(CDD_test, marker="o", color="blue", label='CDI')
plt.plot(CDD_results[1:], marker="o", color="orange", label='CDD')
plt.legend()
plt.show()

In [ ]:
model = ARIMA(CDI_train, order=(1,2,2))
CDI_ARIMA_model = model.fit()
forecast_CDI_ARIMA=CDI_ARIMA_model.forecast(len(CDI_test))
CDI_results = pd.concat([CDI_ARIMA_model.fittedvalues,forecast_CDI_ARIMA])
plt.figure(figsize=(25,8))
plt.plot(CDI_train, marker="o", color="black", label="CDD_train")
plt.plot(CDI_test, marker="o", color="blue", label='CDI')
plt.plot(CDI_results[2:], marker="o", color="orange", label='CDD')
plt.legend()
plt.show()

In [ ]:
forecast_ARIMA = []
for secteur in range(4) :
   
    model = ARIMA(secteur_train[secteur].values, order=(2,1,3)).fit()
    forecast_ARIMA.append(model.forecast(len(secteur_test[secteur])))
    result = (np.concatenate([model.fittedvalues,  forecast_ARIMA[secteur] ]))

    plt.figure(figsize=(25,8))

    plt.plot(secteur_train[secteur].index,secteur_train[secteur].values, marker="o", color="black", label='Secteur'+secteurs[secteur].name)
    plt.plot(secteur_test[secteur].index,secteur_test[secteur].values, marker="o", color="blue", label='CDI')
    plt.plot(secteurs[secteur].index[1:],result[1:], marker="o",color="orange", label='Forecast')
    plt.legend()
    plt.show()

In [ ]:
#Erreur RMSE du modele ARIMA sur le CDD et CDI

rmse_CDD_ARIMA=sqrt(mean_squared_error(CDD_test, forecast_CDD_ARIMA))
rmse_CDI_ARIMA=sqrt(mean_squared_error(CDI_test, forecast_CDI_ARIMA))
#Erreur RMSE du modele ARIMA sur nos 4 secteurs

rmse_arima=[]
for secteur in range(4):
    rmse_arima.append(sqrt(mean_squared_error(secteur_test[secteur], forecast_ARIMA[secteur])))

In [ ]:
#On affiche et compare les RMSE pour la methode de lissage et pour ARIMA
print("RMSE CDD exponential smoothing :",rmse_CDD)
print("RMSE CDD ARIMA :",rmse_CDD_ARIMA)
print("\nRMSE CDI exponential smoothing :",rmse_CDI)
print("RMSE CDI ARIMA :",rmse_CDI_ARIMA)
for secteur in range(4):
    print(f"\nRMSE exponential smoothing {secteurs[secteur].name}:",rmse_[secteur])
    print("RMSE ARIMA {secteurs[secteur].name}:",rmse_arima[secteur])

<h3>Interpretation :</h3>
<ul> 
    <li>DPAE (CDD et CDI) :</li>
    <p>En comparant les valeurs des RMSE respectifs on peut dire que la methode du lissage donne de bien meilleur resultats</p>
    <li>EESSA :</li>
    <ul>
        <li>Secteur 1 : Emplois salariés trimestriels - Industrie - Sections B, C, D et E - CVS :</li>
        <li>Secteur 2 : Emplois salariés trimestriels - Construction - Section F - CVS :</li>
        <li>Secteur 3 : Emplois salariés trimestriels - Tertiaire marchand - Sections G, H, I, J, K, L, M, N, R, S, T - CVS :</li>
        <p>D'après les valeurs des RMSE, pour ces 3 secteurs ARIMA presente une valeur inferieure à celle presentée par le lissage. On en conclut que le ARIMA est mieux adapté</p>
        <li>Secteur 4 : Emplois salariés trimestriels - Industries extractives, énergie, eau, gestion des déchets et dépollution - A17 : DE - CVS :</li>
        <p>Ici les deux erreurs sont faibles, mais le lissage exponentiel l'emporte</p>
    </ul>
</ul>
<p>En general on observe que pour la prediction du DPAE on a de meilleurs resultats à l'aide du lissage exponentiel. Pour l'EESSA on a une meilleure prediciton à l'aide du modèle ARIMA</p>

<h1>Question 6 :</h1>

In [ ]:

model = ARIMA(CDD, order=(2,1,2)).fit()
prediction_CDD= model.forecast(4)
print(prediction_CDD)


model = ARIMA(CDI, order=(1,2,2)).fit()
prediction_CDI= model.forecast(4)
print(prediction_CDI)

prediction_secteur=[]
for secteur in range(4) :
    model = ARIMA(secteurs[secteur].values, order=(2,1,3)).fit()
    prediction_secteur.append(model.forecast(1)[0])


In [ ]:
print(" CDD : ".center(30,"=")+"\nLes predicitons pour fevrier-mai sont :")
print(prediction_CDD)
print(" CDI : ".center(30,"=")+"\nLes predicitons pour fevrier-mai sont :")
print(prediction_CDI)
print("\n")
for secteur in range(4):
    print(f" Secteur {secteur+1} : ".center(30,"=")+f"\nLes predicitons {secteurs[secteur].name} :")
    print(prediction_secteur[secteur])